<a href="https://colab.research.google.com/github/JustJ3di/prompt_eng/blob/main/Finetuning_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Addestra un LLM sui tuoi dati - Tutorial Simone Rizzo
Questo notebook è una demo per dimostrare come si può addestrare un LLM open-source privato sui propri dati per poi esportarlo in GGFU per Ollama.

**Nota bene**: se vuoi usare la GPU gratuita vai da Modifica -> Impostazioni blocco note -> T4 GPU

La demo è stata realizzata da **Simone Rizzo**:
- [Youtube](https://www.youtube.com/channel/UCbMlkb79E12CwveGAtdFj-A)
- [Linkedin](https://www.linkedin.com/in/simone-rizzo-9851b7147/)
- [TikTok](https://www.tiktok.com/@simonerizzo98)
- [Instagram](https://www.instagram.com/simorizzo_ai/)

Seguimi e lascia un like sui miei social 😜

# Installazione delle librerie

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Inizializzazione del modello

In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

NotImplementedError: Unsloth: No NVIDIA GPU found? Unsloth currently only supports GPUs!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.12.4 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "hello"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Hello. How can I assist you today?<|eot_id|>


# Formattazione del testo per il formato di LLama3.2

In [ ]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

In [ ]:
import pandas as pd
df = pd.read_csv("/content/dataset.csv")

In [ ]:
df.head()

,query,response
0,Qual è il settore principale in cui opera Rizz...,Rizzo Company opera principalmente nel settore...
1,Quali sono i tre principali ambiti di utilizzo...,I tre principali ambiti di utilizzo dei prodot...
2,Come si differenzia Rizzo Company nel mercato ...,Rizzo Company si differenzia nel mercato del p...
3,Quali tecnologie all’avanguardia utilizza l’az...,Rizzo Company utilizza diverse tecnologie all’...
4,In che modo Rizzo Company promuove la sostenib...,Rizzo Company promuove la sostenibilità attrav...


In [ ]:
df = df.dropna()

In [ ]:
from datasets import Dataset
df["conversations"] = df.apply(
    lambda x: [
        {"content": x["query"], "role": "user"},
        {"content": x["response"], "role": "assistant"}
    ], axis=1
)

# Ora convertiamo il DataFrame in un Dataset di HuggingFace, rimuovendo le vecchie colonne
dataset = Dataset.from_pandas(df.drop(columns=["query", "response"]))

In [ ]:
dataset['conversations'][0]

[{'content': 'Qual è il settore principale in cui opera Rizzo Company?',
  'role': 'user'},
 {'content': 'Rizzo Company opera principalmente nel settore del packaging sostenibile.',
  'role': 'assistant'}]

# Addestramento del modello

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "conversations",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
6.695 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100 | Num Epochs = 5
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


Step,Training Loss
1,1.531200
2,1.366300
3,1.357700
4,1.346600
5,1.431700
6,1.378100
7,1.240100
8,1.280800
9,1.172700
10,1.250000


# Inferenza con Streaming

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Quali sono i vantaggi competitivi del sistema LTS?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

# Esporto in GGUF per Ollama

In [ ]:
model.save_pretrained_gguf("/content/drive/My Drive/model", tokenizer, quantization_method = "f16")

# Save to q4_k_m GGUF
# model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

# Save to 8bit Q8_0
# model.save_pretrained_gguf("model", tokenizer,)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.57 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 28/28 [00:01<00:00, 18.99it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving /content/drive/My Drive/model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving /content/drive/My Drive/model/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at /content/drive/My Drive/model into f16 GGUF format.
The output location will be /content/drive/My Drive/model/unsloth.F16.gguf
This might take 3 minutes...
usage: convert_hf_to_gguf.py [-h] [--vocab-only] [--outfile OUTFILE]
                             [--outtype {f32,f16,bf16,q8_0,tq1_0,tq2_0,auto}] [--bigendian]
                             [--use-temp-f

RuntimeError: Unsloth: Quantization failed for /content/drive/My Drive/model/unsloth.F16.gguf
You might have to compile llama.cpp yourself, then run this again.
You do not need to close this Python program. Run the following commands in a new terminal:
You must run this in the same folder as you're saving your model.
git clone --recursive https://github.com/ggerganov/llama.cpp
cd llama.cpp && make clean && make all -j
Once that's done, redo the quantization.

In [ ]:
from google.colab import files
files.download('/content/model/unsloth.F16.gguf')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>